In [1]:
import os
import re
import uuid
import csv
import ast
from datetime import datetime

import yaml
import requests
import urllib.request
from urllib.parse import urlparse
from bs4 import BeautifulSoup

import pandas as pd
from openpyxl import load_workbook

In [3]:
def read_yaml_file(file_name):
    with open(file_name, 'r') as file:
        content = yaml.load(file, Loader=yaml.FullLoader)
    return content


In [4]:
yaml_file_path = 'Secret/creadential.yml'

data = read_yaml_file(yaml_file_path)

username = data.get('username', None)
password = data.get('password', None)

print(f"Username: {username}")
print(f"Password: {password}")

Username: franck.kadjo@adisa.digital
Password: 12345678


## AnnuaireParser

In [5]:
class AnnuaireParser:
    def __init__(self, url='https://www.goafricaonline.com/ci/annuaire', parsers=['lxml', 'html.parser']):
        self.base_url = 'https://www.goafricaonline.com/ci/annuaire'
        self.url = url
        self.parsers = parsers
        self.sectors_links = set()
        self.soup = self.init_soup()
        self.current_soup = None
        self.current_link = None
        self.sectors_annuaire_links = set()
        self.category_annuaire_links = set()
        self.company_annuaire_links = set()
        
    def fetch_page(self, url=None):
        if url is None:
            url = self.url
        try:
            response = requests.get(url)
            return response.text
        except Exception as e:
            print(f"Erreur lors de la récupération de la page : {e}")
            return None

    def init_soup(self, url=None):
        html_content = self.fetch_page(url)

        for parser in self.parsers:
            try:
                if isinstance(html_content, str):
                    soup = BeautifulSoup(html_content, parser)
                else:
                    soup = BeautifulSoup(html_content.text, parser)

                return soup

            except Exception as e:
                print(f"Erreur avec le parser {parser}: {e}")

        return None

    def generate_and_save_identifiers(self, csv_filename='gao_identifiers', num_identifiers=1):
        existing_identifiers = set()

        # Lire les identifiants existants dans le fichier CSV
        try:
            with open(f"Secret/{csv_filename}", 'r', newline='') as csvfile:
                reader = csv.DictReader(csvfile)
                for row in reader:
                    existing_identifiers.add(row['Identifiers'])
        except FileNotFoundError:
            pass
    
        identifiers = []
    
        while len(identifiers) < num_identifiers:
            unique_id = 'CI' + str(uuid.uuid4().int)[:8]
    
            # Vérifier si l'identifiant est déjà présent
            if unique_id not in existing_identifiers:
                identifiers.append(unique_id)
                existing_identifiers.add(unique_id)

        # Enregistrer les nouveaux identifiants dans le fichier CSV
        with open(csv_filename, 'a', newline='') as csvfile:
            fieldnames = ['Identifiers']
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    
            if csvfile.tell() == 0:
                writer.writeheader()  # Écrire l'en-tête seulement si le fichier est vide
    
            for identifier in identifiers:
                writer.writerow({'Identifiers': identifier})
                
        return unique_id

    def concat_excel_files(self, folder_path):
        # Liste pour stocker les DataFrames de chaque fichier Excel
        dfs = []
    
        # Parcourir les fichiers dans le dossier
        for filename in os.listdir(folder_path):
            if filename.endswith(".xlsx"):
                file_path = os.path.join(folder_path, filename)
    
                # Lire le fichier Excel dans un DataFrame
                df = pd.read_excel(file_path)
    
                # Ajouter le DataFrame à la liste
                dfs.append(df)
    
        # Concaténer tous les DataFrames dans un seul DataFrame final
        final_df = pd.concat(dfs, ignore_index=True)
    
        return final_df
        
    def extract_text_from_url(self, current_link):
        parsed_url = urlparse(current_link)
        text_from_url = parsed_url.path.split('/')[-1]
        return text_from_url.upper()

    def extract_unique_links(self, link_class="stretched-link text-center", target_url="goafricaonline.com/ci/annuaire"):
        _links_set = set()
        
        if self.current_link:
            self.soup = self.init_soup(self.current_link)
            
        if not self.soup:
            raise ValueError("La soupe n'a pas été initialisée correctement.")
        
        _links = self.soup.find_all('a', attrs={'class': link_class})
        
        for _link in _links:
            if target_url in _link.get("href", ""):
                _links_set.add(_link.get("href"))

        return _links_set

    def get_total_pages(self):
        try:
            self.soup = self.init_soup(self.current_link)
            pagination_tag = self.soup.find('ul', class_='pagination')

            if pagination_tag:
                last_page_tag = pagination_tag.find_all('li')[-2]
                last_page = last_page_tag.text.strip()

                # Extraire uniquement les chiffres de la chaîne
                last_page_number = ''.join(filter(str.isdigit, last_page))

                return int(last_page_number)

        except Exception as e:
            print(f"Erreur lors de la récupération du nombre total de pages : {e}")

        return 0

    def process_annuaire_sectors_links(self):
        self.sectors_links = self.extract_unique_links()
        return list(self.sectors_links)
    
    def get_sectors(self, sectors_links):
        sectors = set()
        
        for sectors_link in sectors_links:
            sectors_name = self.extract_text_from_url(sectors_link)
            sectors.add(sectors_name)
            
        return list(sectors)


    def process_annuaire_category_links(self, sectors_links):
        category_links_data = []
        total_links = 0
    
        for sectors_link in sectors_links:
            self.current_link = sectors_link
            category_links = self.extract_unique_links()
    
            if category_links:
                total_links += len(category_links)
                sector_name = self.extract_text_from_url(sectors_link)
                category_links_dict = {'secteurs': sector_name, 'links': category_links}
                category_links_data.append(category_links_dict)
    
        return total_links, category_links_data



    def extract_company_urls(self):
        total_pages = self.get_total_pages()
        company_urls = []

        if total_pages != 0:
            for page in range(1, total_pages + 1):
                self.current_link = f"{self.current_link}?p={page}"
                try:
                    self.soup = self.init_soup(self.current_link)
                    article_tags = self.soup.find_all('article', {'data-role': 'company'})

                    for article_tag in article_tags:
                        link_tags = article_tag.find_all('a', {'class': 'stretched-link'})
                        for link_tag in link_tags:
                            if 'href' in link_tag.attrs:
                                company_urls.append(link_tag['href'])
                except Exception as e:
                    print(f"Erreur avec le parseur : {e}")

            return company_urls

        return None

    
    def filter_data_by_categories(self, data, categories_to_filter):
        """
        Filtrer les données en fonction des catégories spécifiées.
    
        Parameters:
            data (list): Liste de dictionnaires contenant des catégories et des liens.
            categories_to_filter (list): Liste des catégories à filtrer.
    
        Returns:
            dict: Dictionnaire contenant les données filtrées par catégories.
        """
        filtered_data = {}
    
        for item in data:
            category = item.get('secteurs')
            links = item.get('links')
    
            if category in categories_to_filter:
                filtered_data[category] = links

        if not all(category in filtered_data for category in categories_to_filter):
            missing_categories = [category for category in categories_to_filter if category not in filtered_data]
            raise ValueError(f"Catégories non trouvées : {', '.join(missing_categories)}")
    
        return filtered_data
    
    

    def process_annuaire_company_link(self, category_links, sectors_filter=None, limit=None, full=True):
        company_data = []

        if sectors_key:
            full = False
    
        def process_category_link(category_link, sector_key):
            nonlocal limit
            self.current_link = category_link
            category_name = self.extract_text_from_url(category_link)
            company_links = self.extract_company_urls()
    
            return {
                'secteurs': sector_key,
                'categories': category_name,
                'links': company_links
            }
    
        if full:
            for key, values in category_links.items():
                for category_link in values:
                    if limit is not None and limit > 0:
                        company_data.append(process_category_link(category_link, key))
                        limit -= 1
                    else:
                        company_data.append(process_category_link(category_link, key))
    
            return company_data
    
        if sectors_filter is None:
            raise ValueError("Vous n'avez pas défini de secteur dans la liste Catégorie")
    
        try:
            sectors_category_links = self.filter_data_by_categories(category_links, sectors_filter)
        except ValueError as e:
            print(f"Erreur : {e}")
            
        for sector_key, category_links in sectors_category_links.items():
            for category_link in category_links:
                if limit is not None and limit > 0:
                    company_data.append(process_category_link(category_link, sector_key))
                    limit -= 1
                else:
                    company_data.append(process_category_link(category_link, sector_key))
    
        return company_data if company_data else None


    def try_parsers_find_image(self, balise='img', balise_class=None):
        if balise_class is None:
            balise_class = 'ml-10 w-[118px] h-[118px] object-cover rounded shadow border-solid border-4 border-white cursor-pointer'
            
        try:
            element = self.soup.find(balise, attrs={'class': balise_class})

            if element is not None:
                img_src = element.get('src')
                if img_src:
                    return img_src.strip()
        except Exception as e:
            print(f"Erreur avec le parser : {e}")

        return None

    def try_parsers_find_address(self, balise='address', balise_class='text-14'):
        try:
            address_tag = self.soup.find(balise, attrs={'class': balise_class})

            if address_tag:
                address_text = ' '.join(address_tag.stripped_strings)
                return address_text
            else:
                return None
        except Exception as e:
            print(f"Erreur avec le parser : {e}")
        

    def clean_contact_string(self, contact_string):
        match = re.search(r'\(\+(\d+)\)', contact_string)

        if match:
            indicative = match.group(1)
            cleaned_number = ''.join(char for char in contact_string.split(indicative)[-1] if char.isdigit())
    
            clean_contact = f"+{indicative} {cleaned_number}"
    
            return {'indicatif': f"+{indicative}", 'numero': cleaned_number, 'contact': clean_contact}
    
        return None

    def extract_contact_details(self, balise='div', balise_class='flex mb-4'):
        try:
            contact_div = self.soup.find(balise, class_=balise_class)
            if contact_div is not None:
                a_tag = contact_div.find('a')
                if a_tag:
                    phone_number = a_tag.get_text(strip=True)
                    clean_contact = self.clean_contact_string(phone_number)

                    return clean_contact

        except Exception as e:
            print(f"Erreur avec le parser : {e}")
            return None
            

    def try_parsers_find_website(self, balise='a', balise_class='font-bold text-link block'):
        try:
            element = self.soup.find(balise, attrs={'class': balise_class, 'target':"_blank"})

            if element is not None:
                web_site = element['href'].strip()
                return web_site
        except Exception as e:
            print(f"Erreur avec le parser : {e}")

        return None

    def try_parsers_find_social_links(self, balise='div', balise_class='flex mt-8'):
        try:
            social_divs = self.soup.find_all(balise, attrs={'class': balise_class})

            social_links = {}
            for social_div in social_divs:
                social_classes = {
                    'facebook': 'bg-brand-facebook',
                    'instagram': 'bg-brand-instagram',
                    'linkedin': 'bg-brand-linkedIn',
                    'whatsapp': 'bg-brand-whatsapp'
                }

                for social_type, social_class in social_classes.items():
                    social_elements = social_div.find_all('a', class_=lambda cls: cls and social_class in cls)
                    if social_elements:
                        social_links[social_type] = social_elements[0]['href']
                    else:
                        social_links[social_type] = ''

                return social_links

        except Exception as e:
            print(f"Erreur avec le parser : {e}")

        return {}

    def try_parsers_find_description(self, balise='div', balise_id='description'):
        try:
            description_div = self.soup.find(balise, id=balise_id)

            if description_div is not None:
                return description_div.get_text(strip=True)
        except Exception as e:
            print(f"Erreur avec le parser : {e}")

        return None

    def clean_string_hours(self, text):
        cleaned_text = ' '.join(text.split())
        return cleaned_text

    def extract_opening_hours(self, balise='div', balise_id='planning'):
        opening_hours = {}
        try:
            opening_hours_div = self.soup.find(balise, id=balise_id)

            if opening_hours_div is not None:
                table = opening_hours_div.find('table')
                if table:
                    rows = table.find_all('tr')
                    for row in rows:
                        columns = row.find_all('td')
                        day = columns[0].get_text(strip=True)
                        hours = columns[1].get_text(strip=True)
                        opening_hours[day] = hours

                    cleaned_hours = {day: self.clean_string_hours(hours) for day, hours in opening_hours.items()}

                    return cleaned_hours
        except Exception as e:
            print(f"Erreur avec le parser : {e}")

        return {}


    def extract_services(self, balise='a', balise_class='shopping-product'):
        services = set()
        try:
            service_links = self.soup.find_all(balise, class_=balise_class)

            for link in service_links:
                div_tag = link.find('div', class_='font-bold')
                if div_tag:
                    title = div_tag.text.strip()
                    services.add(title)

        except Exception as e:
            print(f"Erreur avec le parser: {e}")
            
        return list(services)

    def extract_company_text(self):
        try:
            company_element = self.soup.find('h1', attrs={'class': 'text-24 ls:text-32 text-black m-0 mb-3 font-bold leading-snug'})

            if company_element is not None:
                return company_element.text.strip()
        except Exception as e:
            print(f"Erreur avec le parser : {e}")

            return None

        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête HTTP : {e}")
            return None

    def extract_media_links_and_images(self):
        try:
            media_section = self.soup.find('div', {'id': 'medias'})
            if media_section:
                # Trouver tous les éléments li avec la classe iframe-child:w-full
                media_items = media_section.find_all('li', {'class': 'iframe-child:w-full'})
    
                multimedia_data = []
    
                # Parcourir les éléments médias
                for media_item in media_items:
                    # Initialiser un dictionnaire pour stocker les informations multimédias
                    media_info = {}
    
                    # Extraire le lien à partir de la balise a
                    link_tag = media_item.find('a')
                    if link_tag:
                        media_info['link'] = link_tag.get('href')
    
                    # Extraire l'image à partir de la balise img
                    img_tag = media_item.find('img')
                    if img_tag:
                        media_info['image'] = img_tag.get('src')
    
                    # Ajouter les informations multimédias à la liste
                    multimedia_data.append(media_info)
    
                return multimedia_data
            
            return []
        
        except requests.exceptions.RequestException as e:
            print(f"Erreur lors de la requête HTTP : {e}")

    def extract_image_links(self, data_links):
        image_links = {}

        for index, item in enumerate(data_links):
            link = item.get('link')
            image_url = item.get('image')
    
            # Vérifier si le lien est une image et a une URL
            if image_url and link and image_url.startswith('https'):
                image_links[f'image{index+1}'] = image_url
    
        return image_links


    def download_image(self, image_url, download_folder='Medias'):
        if not os.path.exists(download_folder):
            os.makedirs(download_folder)
    
        # Vérifier si l'URL de l'image existe
        if image_url:
            # Extraire le nom du fichier original de l'URL
            filename = os.path.join(download_folder, os.path.basename(image_url))
    
            # Télécharger l'image
            try:
                urllib.request.urlretrieve(image_url, filename)
                print(f"Image téléchargée avec succès : {filename}")
    
                return filename
            except Exception as e:
                print(f"Erreur lors du téléchargement de l'image {filename} : {e}")
        else:
            print("L'URL de l'image est manquante ou invalide.")
            return None
            
                    
    def downloaded_files(self, data_links):
        image_links = self.extract_image_links(data_links)
        
        # Parcourir la liste des données d'images
        for index, url in enumerate(image_links):
            image_url = data.get('image')
            self.download_image(image_url)
            


    def extract_company_info(self, dataframe, sectors=['COMMUNICATION-PUBLICITE'], limit=10):
        
        companies = []
        links_already_visited = []
        data_company_links = dataframe.values.tolist()
        
        for data_company in data_company_links:
            self.current_link = data_company[2]
            self.soup = self.init_soup(self.current_link)

            days_hours = self.extract_opening_hours()
            
            socials_links = self.try_parsers_find_social_links()
            
            multimedia_data = self.extract_media_links_and_images()
            
            if days_hours:
                days_hours = {'Lundi': '08H00 — 17H30',
                              'Mardi': '08H00 — 17H30',
                              'Mercredi': '08H00 — 17H30',
                              'Jeudi': '08H00 — 17H30',
                              'Vendredi': '08H00 — 17H30',
                              'Samedi': 'Fermé',
                              'Dimanche': 'Fermé'}
            
            if len(socials_links) == 0:
                socials_links = {'facebook': '', 'instagram': '', 'linkedin': '', 'whatsapp': '' }

            if multimedia_data:
                image_links = self.extract_image_links(multimedia_data)
            else: 
                image_links = []

            company_info = {
                'identifier': self.generate_and_save_identifiers(),
                'company_secteur': data_company[0],
                'company_categotie': data_company[1],
                'company_adress': self.try_parsers_find_address(),
                'company_name': self.extract_company_text(),
                'company_email': '',
                'company_description': self.try_parsers_find_description(),
                'company_website': self.try_parsers_find_website(),
                'company_url_logo': self.try_parsers_find_image(),
                'company_url': data_company[2],
                'services': self.extract_services(),
                #'medias': image_links
            }
            
            company_info.update(socials_links)
            company_info.update(days_hours)
            company_info.update(image_links)

            companies.append(company_info)
            links_already_visited.append(self.current_link)

        return companies, links_already_visited

    def list_of_dicts_to_dataframe(self, data):
        """
        Convertit une liste de dictionnaires en DataFrame.
        
        Parameters:
            data (list): Liste de dictionnaires à convertir.
        
        Returns:
            pandas.DataFrame: Le DataFrame résultant.
        """
        dataframe = pd.DataFrame(data)
        dataframe = dataframe.explode('links')
        return dataframe
    
    
    def save_dataframe_to_excel(self, dataframe, folder='Resultats', sheet_name=None, filename=None):
        """
        Enregistre un DataFrame dans un fichier Excel avec un nom de fichier personnalisé.
    
        Parameters:
            dataframe (pandas.DataFrame): Le DataFrame à enregistrer.
            sectors_key (str): Clé du secteur pour personnaliser le nom du fichier.
            folder (str): Le dossier où enregistrer le fichier Excel (par défaut, le dossier actuel).
            sheet_name (str): Nom de la feuille dans le fichier Excel (par défaut, utilisera sectors_key en majuscules).
        """
        if sheet_name is None:
            sheet_name = (dataframe.iloc[0, 0]).upper()
    
        timestamp = datetime.now().strftime("%Y%m%d%H%M%S")
        
        if filename is None:
            filename = dataframe.iloc[0, 0]
            
        filepath = f"{filename}_{timestamp}.xlsx"
        
        filepath = os.path.join(folder, filepath)
    
        dataframe.to_excel(filepath, index=False, sheet_name=sheet_name, engine='openpyxl')
    
    def save_dataframe_to_existing_excel(self, dataframe, filepath, sheet_name=None):
        """
        Enregistre un DataFrame dans un fichier Excel existant en spécifiant le nom de la feuille.
    
        Parameters:
            dataframe (pandas.DataFrame): Le DataFrame à enregistrer.
            filepath (str): Chemin vers le fichier Excel existant.
            sheet_name (str): Nom de la feuille dans le fichier Excel (par défaut, utilisera le nom actuel).
        """
        if sheet_name is None:
            sheet_name = 'Sheet1'  # Modifiez ceci avec le nom actuel de votre feuille si nécessaire
    
        with pd.ExcelWriter(filepath, engine='openpyxl') as writer:
            writer.book = load_workbook(filepath)
            dataframe.to_excel(writer, index=False, sheet_name=sheet_name)

In [6]:
 def calculate_company_links_length(company_data):
    company_links_length = 0
    for index in range(len(company_data)):
        company_links_length += len(company_data[index]['links'])
    return company_links_length

## Begin Text

In [7]:
annuaire_parser = AnnuaireParser()

In [8]:
sectors_links = annuaire_parser.process_annuaire_sectors_links()
sectors_links

['https://www.goafricaonline.com/ci/annuaire/professionnels-immobilier',
 'https://www.goafricaonline.com/ci/annuaire/maison-decoration',
 'https://www.goafricaonline.com/ci/annuaire/entreprises-finances',
 'https://www.goafricaonline.com/ci/annuaire/urgence',
 'https://www.goafricaonline.com/ci/annuaire/entreprises-batiment-construction',
 'https://www.goafricaonline.com/ci/annuaire/administrations',
 'https://www.goafricaonline.com/ci/annuaire/telecommunications',
 'https://www.goafricaonline.com/ci/annuaire/informatique-internet',
 'https://www.goafricaonline.com/ci/annuaire/mode-vetements-textiles',
 'https://www.goafricaonline.com/ci/annuaire/produits-hygiene',
 'https://www.goafricaonline.com/ci/annuaire/sport',
 'https://www.goafricaonline.com/ci/annuaire/autres',
 'https://www.goafricaonline.com/ci/annuaire/entreprises-agroalimentaires',
 'https://www.goafricaonline.com/ci/annuaire/societes-securite',
 'https://www.goafricaonline.com/ci/annuaire/bien-etre',
 'https://www.goafri

In [9]:
len(sectors_links)

31

In [10]:
sectors = annuaire_parser.get_sectors(sectors_links)

In [11]:
sectors

['PRODUITS-HYGIENE',
 'ARTISANS',
 'MAISON-DECORATION',
 'COMPTABILITE-JURIDIQUE-CONSEIL',
 'COMMERCES',
 'ENTREPRISES-AGROALIMENTAIRES',
 'INFORMATIQUE-INTERNET',
 'SOCIETES-SECURITE',
 'SECTEUR-ENERGIE',
 'TELECOMMUNICATIONS',
 'MODE-VETEMENTS-TEXTILES',
 'FORMATIONS-EDUCATION',
 'ALIMENTATION',
 'SPORT',
 'BIEN-ETRE',
 'EMPLOI',
 'URGENCE',
 'ENTREPRISES-FINANCES',
 'TRANSPORTS',
 'TOURISME-ET-LOISIRS',
 'PROFESSIONNELS-IMMOBILIER',
 'INDUSTRIES',
 'COMMUNICATION-PUBLICITE',
 'SANTE',
 'AUTRES',
 'ASSOCIATIONS-PROFESSIONNELLES',
 'SOCIETES-SERVICES',
 'ENTREPRISES-BATIMENT-CONSTRUCTION',
 'AUTOMOBILE-MOTO',
 'COMMERCE-IMPORT-EXPORT',
 'ADMINISTRATIONS']

In [12]:
df_sectors = pd.DataFrame(sectors)

In [13]:
df_sectors.to_csv('liste_secteurs.csv')

In [73]:
total_links, category_links = annuaire_parser.process_annuaire_category_links(sectors_links)

In [74]:
total_links

513

In [75]:
category_links

[{'secteurs': 'INDUSTRIES',
  'links': {'https://www.goafricaonline.com/ci/annuaire/air-comprime-accessoires',
   'https://www.goafricaonline.com/ci/annuaire/armes-munitions',
   'https://www.goafricaonline.com/ci/annuaire/boulonnerie-robinetterie',
   'https://www.goafricaonline.com/ci/annuaire/brasseries',
   'https://www.goafricaonline.com/ci/annuaire/cartonnerie',
   'https://www.goafricaonline.com/ci/annuaire/cat-industries',
   'https://www.goafricaonline.com/ci/annuaire/cimenterie',
   'https://www.goafricaonline.com/ci/annuaire/coton-production-exploitation',
   'https://www.goafricaonline.com/ci/annuaire/depannage-maintenance-industrielle',
   'https://www.goafricaonline.com/ci/annuaire/emballage-conditionnement',
   'https://www.goafricaonline.com/ci/annuaire/entreprises-froid-industriel',
   'https://www.goafricaonline.com/ci/annuaire/equipements-industriels',
   'https://www.goafricaonline.com/ci/annuaire/etiquetage-tracabilite',
   'https://www.goafricaonline.com/ci/annuai

In [76]:
df_category = annuaire_parser.list_of_dicts_to_dataframe(category_links)

In [77]:
df_category.head()

,secteurs,links
0,INDUSTRIES,https://www.goafricaonline.com/ci/annuaire/exp...
0,INDUSTRIES,https://www.goafricaonline.com/ci/annuaire/tuiles
0,INDUSTRIES,https://www.goafricaonline.com/ci/annuaire/fab...
0,INDUSTRIES,https://www.goafricaonline.com/ci/annuaire/air...
0,INDUSTRIES,https://www.goafricaonline.com/ci/annuaire/ind...


In [222]:
annuaire_parser.save_dataframe_to_excel(df_category)

In [374]:
sectors_key = [sectors[30]]
print(sectors_key)

['SECTEUR-ENERGIE']


In [369]:
company_data = annuaire_parser.process_annuaire_company_link(category_links, sectors_key)
len(company_data), calculate_company_links_length(company_data)

(13, 1370)

In [370]:
company_data[0]

{'secteurs': 'SECTEUR-ENERGIE',
 'categories': 'ELECTRICITE-PRODUCTION-DISTRIBUTION',
 'links': ['https://www.goafricaonline.com/ci/723970-holding-hilmann-enes-batiment-travaux-publics-abidjan-cote-ivoire',
  'https://www.goafricaonline.com/ci/791362-avtomat-group',
  'https://www.goafricaonline.com/ci/368101-abc-contracting-electricite-production-distribution-abidjan-cote-ivoire',
  'https://www.goafricaonline.com/ci/781983-africa-cooling-service-electricite-production-distribution-abidjan-cote-ivoire',
  'https://www.goafricaonline.com/ci/502851-afrique-forages-hydrauliques-electricite-production-distribution-abidjan-cote-ivoire',
  'https://www.goafricaonline.com/ci/62354-aggreko-electricite-abidjan-cote-ivoire',
  'https://www.goafricaonline.com/ci/169133-ai-entreprise-electricite-abidjan-cote-ivoire',
  'https://www.goafricaonline.com/ci/514150-alynimo-electricite-abidjan-cote-ivoire',
  'https://www.goafricaonline.com/ci/826793-anare-ci-electricite-production-distribution-abidjan

In [371]:
df_company = annuaire_parser.list_of_dicts_to_dataframe(company_data)
df_company.head(10)

,secteurs,categories,links
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/723970-holdi...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/791362-avtom...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/368101-abc-c...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/781983-afric...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/502851-afriq...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/62354-aggrek...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/169133-ai-en...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/514150-alyni...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/826793-anare...
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/585027-aveni...


In [372]:
annuaire_parser.save_dataframe_to_excel(df_company)

In [410]:
df_text = df_company[:1]
df_text.head()

,secteurs,categories,links
0,SECTEUR-ENERGIE,ELECTRICITE-PRODUCTION-DISTRIBUTION,https://www.goafricaonline.com/ci/723970-holdi...


In [381]:
data = df_text.values.tolist()
data

[['SECTEUR-ENERGIE',
  'ELECTRICITE-PRODUCTION-DISTRIBUTION',
  'https://www.goafricaonline.com/ci/723970-holding-hilmann-enes-batiment-travaux-publics-abidjan-cote-ivoire'],
 ['SECTEUR-ENERGIE',
  'ELECTRICITE-PRODUCTION-DISTRIBUTION',
  'https://www.goafricaonline.com/ci/791362-avtomat-group'],
 ['SECTEUR-ENERGIE',
  'ELECTRICITE-PRODUCTION-DISTRIBUTION',
  'https://www.goafricaonline.com/ci/368101-abc-contracting-electricite-production-distribution-abidjan-cote-ivoire'],
 ['SECTEUR-ENERGIE',
  'ELECTRICITE-PRODUCTION-DISTRIBUTION',
  'https://www.goafricaonline.com/ci/781983-africa-cooling-service-electricite-production-distribution-abidjan-cote-ivoire'],
 ['SECTEUR-ENERGIE',
  'ELECTRICITE-PRODUCTION-DISTRIBUTION',
  'https://www.goafricaonline.com/ci/502851-afrique-forages-hydrauliques-electricite-production-distribution-abidjan-cote-ivoire'],
 ['SECTEUR-ENERGIE',
  'ELECTRICITE-PRODUCTION-DISTRIBUTION',
  'https://www.goafricaonline.com/ci/62354-aggreko-electricite-abidjan-cote-i

## Get Data Inforamtion

In [422]:
raison_social

[{'company_secteur': 'SECTEUR-ENERGIE',
  'company_categotie': 'ELECTRICITE-PRODUCTION-DISTRIBUTION',
  'company_adress': 'Quartier GFCI , rue Dimbokro, Non loin de ADVANS\nMarcory - 01 BP 12594 Abidjan 01\nAbidjan - Côte d’Ivoire',
  'company_social_media_links': None,
  'company_url_logo': 'https://www.goafricaonline.com/media/cache/resolve/w200/uploads/media/company_square/0003/39/64020e7f47dac-logo-holding-hilmann-abidjan-cote-ivoire.jpg',
  'company_url': 'https://www.goafricaonline.com/ci/723970-holding-hilmann-enes-batiment-travaux-publics-abidjan-cote-ivoire'}]

In [22]:
category = try_parsers_find(html_company_page, 'a', 'text-16 ls:text-20 text-link w-full')
category

'Agences de communication'

In [23]:
company_logo = try_parsers_find_image(html_company_page, 'img', 'ml-10 w-[118px] h-[118px] object-cover rounded shadow border-solid border-4 border-white cursor-pointer')
company_logo

'https://www.goafricaonline.com/media/cache/resolve/w200/uploads/media/company_square/0002/76/623a0f1919047-LOGO ASENSIA.jpg'

In [24]:
address = try_parsers_find_address(html_company_page, 'address', 'text-14')
address

'Rue Louis Lumière, Zone 4\nMarcory\nAbidjan - Côte d’Ivoire'

In [25]:
contacts = try_parsers_find_contact(html_company_page, 'a', 'text-13 text-gray-700 underline hover:no-underline')
contacts

In [26]:
website = try_parsers_find_website(html_company_page, 'a', 'font-bold text-link block')
website

In [40]:
social_links = try_parsers_find_facebook_links(html_company_page)
social_links

{'facebook': 'https://www.facebook.com/Asensia',
 'instagram': 'https://www.instagram.com/asensia_africa/',
 'linkedin': 'https://www.linkedin.com/company/asensia-africa/',
 'whatsapp': 'https://api.whatsapp.com/send?phone=2250701508398&text=Bonjour%2C%20je%20vous%20contacte%20de%20la%20part%20de%20%2AGo%20Africa%20Online%2A%2C%0A'}

In [56]:
description = try_parsers_find_description(html_company_page, 'div', 'description')

if description:
    print(description)
else:
    print("Aucun élément trouvé avec les parsers spécifiés.")

AGENCE ENDÉVELOPPEMENT DE MARQUESASENSIA AFRICA GROUP, agence conseil en stratégie de développement de Marques, s'érige en moteur d'ascension pour les entreprises responsables en Afrique. Notre expertise unique en Relations Publics 2.0 Digital réinvente & construit des réputations solides et des images influentes, d’abord en interne puis vers l’externe.En développant le sens et la raison d'être des organisations, elle revitalise l’ADN et la culture d'entreprise, stimulant ainsi la raison d’être et l’intelligence collective du capital humain, renforcé par des ateliers de développement de potentiel et de parcours SENS+ agréés par le FDFP (Fonds de Développement de la Formation Professionnelle). Avec des contenus spécifiques et des événements BtoB sur mesure, ASENSIA garantit une visibilité exceptionnelle grâce aux médias & à la force de ses réseaux.Guidée par ses valeurs de confiance, d'empathie et d'authenticité, ASENSIA se positionne comme l'allié indispensable des équipes dirigeantes 

In [54]:
html_test = '''
<div id="description">
    <div class="p-10 border-b border-gray-300 last:border-none overflow-x-auto">
        <div class="colored-titles bbcode-type bbcode-type-company_presentation text-14 ls:text-16 leading-relaxed">
            <br>
            IBAC Aide les entreprises et les particuliers à se développer pour profiter des avantages du web marketing. Nous vous accompagnons dans l’amélioration de vos compétences en marketing digital (social media marketing, formation…).<br>
            Notre objectif est d’aider les entreprises à développer leur activité grâce à Internet. Nous proposons des services de création de site internet, de gestion de compte sur les réseaux sociaux, d’infographie, de publicité internet, de stratégie et de développement digital. Nos services visent à augmenter votre visibilité, améliorer votre notoriété et augmenter votre chiffre d’affaires.
        </div>
    </div>
</div>
'''


In [55]:
description = try_parsers_find_description(html_test, 'div', 'description')

if description:
    print(description)
else:
    print("Aucun élément trouvé avec les parsers spécifiés.")

IBAC Aide les entreprises et les particuliers à se développer pour profiter des avantages du web marketing. Nous vous accompagnons dans l’amélioration de vos compétences en marketing digital (social media marketing, formation…).Notre objectif est d’aider les entreprises à développer leur activité grâce à Internet. Nous proposons des services de création de site internet, de gestion de compte sur les réseaux sociaux, d’infographie, de publicité internet, de stratégie et de développement digital. Nos services visent à augmenter votre visibilité, améliorer votre notoriété et augmenter votre chiffre d’affaires.


In [68]:
opening_hours = extract_opening_hours(html_company_page)
opening_hours

{'Mardi': '08H00 — 17H30',
 'Mercredi': '08H00 — 17H30',
 'Jeudi': '08H00 — 17H30',
 'Vendredi': '08H00 — 17H30',
 'Samedi': 'Fermé',
 'Dimanche': 'Fermé'}

In [102]:
cleaned_contact = extract_contact_details(html_company_page, 'div', 'flex mb-4')
cleaned_contact

{'indicatif': '+225', 'numero': '0701508398', 'contact': '+225 0701508398'}

In [109]:
services_list = extract_services(html_company_page)
services_list

['Cultures d’entreprise',
 'Les événements SENS+',
 'Innovation managériale',
 'Image de marque externe & interne']

## Use Excel File

In [18]:
path_file = "Resultats/ADMINISTRATIONS_20240223150907.xlsx"

In [19]:
df_raw = pd.read_excel(path_file)
df_raw.head()

,secteurs,categories,links
0,ADMINISTRATIONS,MINISTERES,https://www.goafricaonline.com/ci/821118-termi...
1,ADMINISTRATIONS,MINISTERES,https://www.goafricaonline.com/ci/821123-cgi-g...
2,ADMINISTRATIONS,MINISTERES,https://www.goafricaonline.com/ci/821121-cgi-t...
3,ADMINISTRATIONS,MINISTERES,https://www.goafricaonline.com/ci/821120-direc...
4,ADMINISTRATIONS,MINISTERES,https://www.goafricaonline.com/ci/821122-guich...


In [20]:
df = df_raw.copy()

In [21]:
df.shape

(1747, 3)

In [22]:
companies = []
links_visited = []
companies, links_visited = annuaire_parser.extract_company_info(df[:5])

In [23]:
companies

[{'company_secteur': 'ADMINISTRATIONS',
  'company_categotie': 'MINISTERES',
  'company_adress': '8X7H+R78 - Plateau\nAbidjan\nAbidjan - Côte d’Ivoire',
  'company_name': "Terminal Fruitier d'Abidjan",
  'company_email': '',
  'company_description': None,
  'company_houre': {'Mardi': '00H00 — 00H00',
   'Mercredi': '00H00 — 00H00',
   'Jeudi': '00H00 — 00H00',
   'Vendredi': '00H00 — 00H00',
   'Samedi': '00H00 — 00H00',
   'Dimanche': '00H00 — 00H00'},
  'company_social_media_links': None,
  'company_website': None,
  'company_url_logo': None,
  'company_url': 'https://www.goafricaonline.com/ci/821118-terminal-fruitier-d-abidjan',
  'services': None},
 {'company_secteur': 'ADMINISTRATIONS',
  'company_categotie': 'MINISTERES',
  'company_adress': 'GGM8+WR5 Quartier Residence\nGuiglo - Côte d’Ivoire',
  'company_name': 'CGI GUIGLO',
  'company_email': '',
  'company_description': None,
  'company_houre': None,
  'company_social_media_links': None,
  'company_website': None,
  'company_

In [24]:
links_visited

['https://www.goafricaonline.com/ci/821118-terminal-fruitier-d-abidjan',
 'https://www.goafricaonline.com/ci/821123-cgi-guiglo',
 'https://www.goafricaonline.com/ci/821121-cgi-treichville',
 'https://www.goafricaonline.com/ci/821120-direction-generale-des-affaires-maritimes-et-portuaires',
 'https://www.goafricaonline.com/ci/821122-guichet-unique-automobile-de-bouake']

## Use All File

In [91]:
folder_path = 'Resultats'
df_raw = annuaire_parser.concat_excel_files(folder_path)

In [92]:
df_raw.head()

,secteurs,categories,links
0,TRANSPORTS,TRANSPORTS-INTERNATIONAUX,https://www.goafricaonline.com/ci/828280-gcl-t...
1,TRANSPORTS,TRANSPORTS-INTERNATIONAUX,https://www.goafricaonline.com/ci/74438-gls-tr...
2,TRANSPORTS,TRANSPORTS-INTERNATIONAUX,https://www.goafricaonline.com/ci/542548-group...
3,TRANSPORTS,TRANSPORTS-INTERNATIONAUX,https://www.goafricaonline.com/ci/133784-k-log...
4,TRANSPORTS,TRANSPORTS-INTERNATIONAUX,https://www.goafricaonline.com/ci/504986-maind...


In [93]:
df_raw.shape

(86514, 3)

In [123]:
df_raw.to_excel('ALL_COMPAGNIES_URL.xlsx', index=False, engine='openpyxl')

In [94]:
df = df_raw.copy()

In [7]:
path_file_com = 'Resultats/COMMUNICATION-PUBLICITE_20240223134713.xlsx'

In [8]:
df_raw_com = pd.read_excel(path_file_com)

In [9]:
df_raw_com.shape

(5554, 3)

In [10]:
df = df_raw_com.copy()

In [99]:
companies, links_visited = annuaire_parser.extract_company_info(df.iloc[2005:2055])

In [100]:
companies

[{'identifier': 'CI30135218',
  'company_secteur': 'COMMUNICATION-PUBLICITE',
  'company_categotie': 'IMPRIMERIES',
  'company_adress': 'En face de l’agence MTN et la SGBCI - Riviéra Jardins Cocody - 26 BP 451 Abidjan 26 Abidjan - Côte d’Ivoire',
  'company_name': 'ALY CARTOUCHE INFORMATIQUE',
  'company_email': '',
  'company_description': 'ALY CARTOUCHE INFORMATIQUE est à votre disposition pour tous travaux dans le domaine de l’imprimerie. Nous sommes spécialisés dans les domaines suivants :Plaques jusqu’à 300gr, Posters, Cartes postales, Chemises jusqu’à 300gr, Cartes de visite, Cartes commerciales, Cartes de correspondance, Brochures dépliantes, Calques, Etiquettes adhésives et transparentes, Brochures, Calendriers, Couvertures de livre, Packagings, Routages mailing, Finition, Faire-parts, Affiches grand Format, Tampons et Plaques gravées, l’Impression et la Photocopie.',
  'company_website': None,
  'company_url_logo': 'https://www.goafricaonline.com/media/cache/resolve/w200/uploa

In [101]:
len(companies)

50

In [122]:
df_companies = pd.DataFrame(companies)

In [123]:
df_companies.head()

,identifier,company_secteur,company_categotie,company_adress,company_name,company_email,company_description,company_website,company_url_logo,company_url,...,image21,image22,image23,image24,image25,image26,image27,image28,image29,image30
0,CI30135218,COMMUNICATION-PUBLICITE,IMPRIMERIES,En face de l’agence MTN et la SGBCI - Riviéra ...,ALY CARTOUCHE INFORMATIQUE,,ALY CARTOUCHE INFORMATIQUE est à votre disposi...,None,https://www.goafricaonline.com/media/cache/res...,https://www.goafricaonline.com/ci/74391-aly-ca...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,CI63564825,COMMUNICATION-PUBLICITE,IMPRIMERIES,"Face Lycée français Blaise Pascal, Coprim - Ri...",L'ATELIER,,L’Atelier est une imprimerie de 300 m² à la ri...,None,https://www.goafricaonline.com/media/cache/res...,https://www.goafricaonline.com/ci/93607-atelie...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,CI20065501,COMMUNICATION-PUBLICITE,IMPRIMERIES,Près de l’Agence SIB - Siporex Yopougon - 03 B...,IMPRIMERIE LOYAUTE,,L'IMPRIMERIE LOYAUTE est une imprimerie spécia...,None,https://www.goafricaonline.com/media/cache/res...,https://www.goafricaonline.com/ci/112401-loyau...,...,https://www.goafricaonline.com/media/cache/res...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,CI22931966,COMMUNICATION-PUBLICITE,IMPRIMERIES,Rue D33 - Ambassade des USA - Riviera Golf Coc...,SOLUCIOL IMPRIMERIE,,SOLUCIOL prend en charge votre commande depuis...,https://soluciol.com/,https://www.goafricaonline.com/media/cache/res...,https://www.goafricaonline.com/ci/109241-soluc...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,CI15732640,COMMUNICATION-PUBLICITE,IMPRIMERIES,"Angré , pharmacie des allées Cocody Abidjan - ...",OTHENTICK ENTREPRISE,,OTHENTICK structure de prestations de services...,None,https://www.goafricaonline.com/media/cache/res...,https://www.goafricaonline.com/ci/649667-othen...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [104]:
df_companies.to_excel('Resultats/compagnies_data_test.xlsx', index=False, engine='openpyxl')

In [125]:
# Liste pour stocker les valeurs extraites
valeurs_extraites = []

# Boucle pour parcourir chaque élément de la colonne 'services'
for service in df_companies['services']:
    if isinstance(service, list):  # Vérifier si c'est une liste
        valeurs_extraites.extend(service)  # Ajouter les éléments à la liste
    elif isinstance(service, str):  # Vérifier si c'est une chaîne de caractères
        # Extraire les éléments et les ajouter à la liste
        valeurs_extraites.extend([s.strip() for s in service.strip('[]').split(',') if s.strip()])  

# Supprimer les doublons et conserver l'ordre
valeurs_extraites = list(set(valeurs_extraites))

# Afficher la liste des valeurs extraites
valeurs_extraites


['Publicité sur Lieu de Vente (PLV)',
 'Factures normalisée',
 'Bloc note',
 'Sacs cabas personnalisés',
 'Calendrier',
 'Affiche',
 'Tous travaux de polyester',
 'Etui de savon',
 'Construction Métallique',
 'Impression sur T.Shirt',
 'Branding de véhicules  toutes catégories',
 'Kakémono',
 'Carnet de facture',
 'Gadgets',
 'La sérigraphie sur gadgets publicitaires',
 'Carte de visite',
 'Impressions Numériques',
 'Impression numérique',
 'Cartes de visite',
 'Prospectus',
 'Confection de sac Cabas',
 'Infographie (création graphique)',
 'Impressions de haute qualité (offset et numérique)',
 'Décorations publicitaires indoor et outdoor',
 "Tout type d'Impression",
 'Enseignes lumineuses',
 'Impression de tee-shirt']